In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as si
import chameleon as c

I want to use the scipy integrate method `solve_ivp` to solve a system of differential equations.

In [ ]:
m = 1
L = 1
cd = 2
a = 1
E = 1
T = 10

In [ ]:
def sig_a(env, t):
    return -env.pos_init*t

def integrate_to_t(env, t):
    """Integrate the environment from 0 to t"""
    time_steps = int(t / env.dt)
    t_arr = np.linspace(0, t, time_steps)
    for t in t_arr:
        sig = sig_a(env, t)
        env.one_step_return(sig)
    return

env = c.Chameleon(dt=0.1, c = cd)
integrate_to_t(env, T)

In [ ]:
env.u_current

In [ ]:
plt.plot(env.pos_init, env.u_current)

In [ ]:
def G(t, L):
    env = c.Chameleon(init_length=L)
    sig_a = -env.pos_init * t
    dx = env.pos_init[1] - env.pos_init[0]
    g = -si.trapezoid(sig_a, dx=dx)
    return g


def system(t, y, m, L, c, a, E, G_fun):
    """simulate dy/dt = -y"""
    mat = np.array([[0, 1], [-E / (L * m), -(L * c + a) / (L * m)]])
    y_out = np.matmul(mat, y) + np.array([0, G_fun(t,L) / (L * m)])
    return y_out

sol = si.solve_ivp(system, [0, T], np.array([0, 0]), args=[m, L, cd, a, E, G])

In [ ]:
y1 = sol["y"][0]
y2 = sol["y"][1]
t = sol["t"]
F = -cd*y2 - m*np.gradient(y2, t, edge_order=2) 

In [ ]:
t.shape

In [ ]:
# analytic = [{0.},{-0.422633},{-0.76084},{-1.04321},{-1.27968},{-1.47777},{-1.6437},{-1.7827},{-1.89914},{-1.99667},{-2.07837}]
# a = [s.pop() for s in analytic]
# trange = np.linspace(0, T, len(a))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(t, y1)
plt.plot(t, y2)
plt.plot(t, F)
plt.plot(np.linspace(0, T, len(env.F)), env.F)
# plt.plot(trange, a)
plt.xlabel("t")
plt.ylabel("$u_L(t)$")
plt.legend(("$u_L$", r"$\frac{du_L}{dt}$", r"$F(t)$", "F from sim"))
plt.show()

In [ ]:
env.u_current